<a href="https://colab.research.google.com/github/srdiegorodrigues/projeto/blob/main/classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
#Importação das bibliotecas
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import io
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [9]:
#Carregar arquivos a serem analisados
url = "/content/drive/MyDrive/MESTRADO/DADOS/OCORRENCIAS/"

df_2017 = pd.read_csv(io.StringIO(open(f'{url}datatran2017.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2018 = pd.read_csv(io.StringIO(open(f'{url}datatran2018.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2019 = pd.read_csv(io.StringIO(open(f'{url}datatran2019.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2020 = pd.read_csv(io.StringIO(open(f'{url}datatran2020.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2021 = pd.read_csv(io.StringIO(open(f'{url}datatran2021.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
df_2022 = pd.read_csv(io.StringIO(open(f'{url}datatran2022.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)

df_2017 = df_2017.dropna()
df_2018 = df_2018.dropna()
df_2019 = df_2019.dropna()
df_2020 = df_2020.dropna()
df_2021 = df_2021.dropna()
df_2022 = df_2022.dropna()

dfs = [df_2017, df_2018, df_2019, df_2020, df_2021, df_2022]

<ipython-input-9-5849736c2be4>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_2017 = pd.read_csv(io.StringIO(open(f'{url}datatran2017.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
<ipython-input-9-5849736c2be4>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_2018 = pd.read_csv(io.StringIO(open(f'{url}datatran2018.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
<ipython-input-9-5849736c2be4>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_2019 = pd.read_csv(io.StringIO(open(f'{url}datatran2019.csv', 'r', encoding='ISO-8859-1').read()), sep=';', error_bad_lines =False)
<ipython-input-9-5849736c2be4>:7: FutureWarning: The error_bad_lines a

In [10]:
#carregar base de dados dos feriados nacionais
df_feriado = pd.read_csv("/content/drive/MyDrive/MESTRADO/DADOS/OCORRENCIAS/feriados_nacionais.csv",  encoding='ISO-8859-1', sep=";")

#arredondar o valor de km para cima ou para baixo, conforme o valor após o ponto
def arredondar_km(valor):
    valor_float = float(valor)  # Converter para float
    inteiro = int(valor_float)  # parte inteira do valor
    decimal = valor_float - inteiro  # parte decimal do valor

    # Se o valor decimal for maior ou igual a 0.5, arredonda para cima, caso contrário, arredonda para baixo
    if decimal >= 0.5:
        return inteiro + 1
    else:
        return inteiro

#converter o valor da feature br para int
def br_int(valor):
    valor_float = float(valor)
    valor_int = int(valor_float)
    return valor_int

dias_semana_para_int = {
        'segunda-feira': 0,
        'terça-feira': 1,
        'quarta-feira': 2,
        'quinta-feira': 3,
        'sexta-feira': 4,
        'sábado': 5,
        'domingo': 6
    }

def converter_dia_semana_para_int(dia):
    return dias_semana_para_int[dia]

for df in dfs:
    #alterar condições metereologicas para evitar outliers
    df['condicao_metereologica'] = df['condicao_metereologica'].str.replace('Ignorado', 'Céu Claro')
    df['condicao_metereologica'] = df['condicao_metereologica'].str.replace('Neve', 'Chuva')

    #substituir / por - na data_inversa
    df['data_inversa'] = df['data_inversa'].str.replace('/', '-')

    #criação da feature chuva para determinar se a pista estava molhada na hora do acidente
    for indice, item in df.iterrows():
        if item['condicao_metereologica'] in ['Chuva', 'Garoa/Chuvisco', 'Granizo']:
            df.at[indice, 'chuva'] = 1
        else:
            df.at[indice, 'chuva'] = 0

    #fragmentação da feature data_inversa em dia, mes e ano
    df[['dia', 'mes', 'ano']] = df['data_inversa'].apply(lambda x: pd.Series(x.split('-')).astype(int))

    # #Concatenar colunas data_inversa e horario e criando a feature data_hora
    df['data_hora'] = df.apply(lambda x: '%s-%s' % (x['data_inversa'],x['horario']), axis=1)
    #Criação da feature data_hora_int
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    # Convertendo as datas para timestamps Unix
    df['data_hora_int'] = df['data_hora'].astype(int) // 10**9

    #normalizar a feature data_hora_int
    scaler = MinMaxScaler()
    df['data_hora_int'] = scaler.fit_transform(df['data_hora_int'].values.reshape(-1, 1))

    #substituir a , por . na feature km
    df['km'] = df['km'].str.replace(',','.')
    df['km'] = df['km'].apply(arredondar_km)
    df['br'] = df['br'].apply(br_int)


    df['dia_semana_int'] =  df['dia_semana'].apply(converter_dia_semana_para_int)

    #Setar se dia da ocorrência do acidente era feriado ou não
    #Criando a feature feriado com base nas portarias do governo federal
    #Caso seja feriado, será atribuido o valor 1 para o registro
    #caso contrário, será atribuido o valor 0
    df['data_inversa'] = pd.to_datetime(df['data_inversa'])
    df_feriado['Data'] = pd.to_datetime(df_feriado['Data'])
    df_feriado['Data'] = df_feriado['Data'].sort_values(ascending=False).values
    # Criando uma nova coluna 'feriado' no DataFrame principal e inicializando com 0
    df['feriado'] = 0

    # Loop for para comparar as datas e atribuir 1 se forem iguais
    for index, row in df.iterrows():
        if row['data_inversa'] in df_feriado['Data'].values:
            df.at[index, 'feriado'] = 1

<ipython-input-10-74175e345fe5>:76: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df['data_inversa'] = pd.to_datetime(df['data_inversa'])
<ipython-input-10-74175e345fe5>:77: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df_feriado['Data'] = pd.to_datetime(df_feriado['Data'])


In [11]:
#features que serão usadas para execução dos modelos

# Selecionar as colunas categóricas
from sklearn.preprocessing import LabelEncoder, StandardScaler

cols_categoricas = ['uf','condicao_metereologica','delegacia','uop']

# Aplicar a codificação LabelEncoder em cada coluna categórica
label_encoder = LabelEncoder()
for df in dfs:
    for col in cols_categoricas:
        df[col] = label_encoder.fit_transform(df[col])

for df in dfs:
    df['regional_int'] = label_encoder.fit_transform(df['regional'])

colunas = ['km', 'uf', 'br', 'condicao_metereologica', 'regional','regional_int', 'delegacia', 'uop','chuva','dia','mes','ano','feriado', 'dia_semana_int']

In [12]:
#base de dados para algoritmos de regressão e classificação
treinamento1 = pd.concat([df_2017, df_2018, df_2019, df_2020, df_2021], ignore_index=True) #TRAIN_RAW
teste1 = df_2022[colunas] #TEST_RAW

#dados para teste
treinamento1 = treinamento1[colunas]
sr_df_r = treinamento1[treinamento1['regional'] == 'SPRF-DF']
sr_df_br_020_r = sr_df_r[sr_df_r['br'] == 20]
sr_df_br_040_r = sr_df_r[sr_df_r['br'] == 40]
sr_sp_r = treinamento1[treinamento1['regional'] =='SPRF-SP']
sr_sp_br_101_r = sr_sp_r[sr_sp_r['br'] == 101]
sr_sp_br_116_r = sr_sp_r[sr_sp_r['br'] == 116]
sr_sc_r = treinamento1[treinamento1['regional'] =='SPRF-SC']
sr_sc_br_101_r = sr_sc_r[sr_sc_r['br'] == 101]
sr_sc_br_116_r = sr_sc_r[sr_sc_r['br'] == 116]

#dados para teste
sr_df_t_r = teste1[teste1['regional'] == 'SPRF-DF']
sr_df_br_020_t_r = sr_df_t_r[sr_df_t_r['br'] == 20]
sr_df_br_040_t_r = sr_df_t_r[sr_df_t_r['br'] == 40]
sr_sp_t_r = teste1[teste1['regional'] =='SPRF-SP']
sr_sp_br_101_t_r = sr_sp_t_r[sr_sp_t_r['br'] == 101]
sr_sp_br_116_t_r = sr_sp_t_r[sr_sp_t_r['br'] == 116]
sr_sc_t_r = teste1[teste1['regional'] =='SPRF-SC']
sr_sc_br_101_t_r = sr_sc_t_r[sr_sc_t_r['br'] == 101]
sr_sc_br_116_t_r = sr_sc_t_r[sr_sc_t_r['br'] == 116]

regionais = teste1['regional'].unique()

#bases de dados que serão utilizadas
df_treinamento_r = [sr_df_r, sr_df_br_020_r, sr_df_br_040_r, sr_sp_r, sr_sp_br_101_r,
                  sr_sp_br_116_r, sr_sc_r, sr_sc_br_101_r, sr_sc_br_116_r]
df_teste_r = [sr_df_t_r, sr_df_br_020_t_r, sr_df_br_040_t_r,sr_sp_t_r, sr_sp_br_101_t_r,
            sr_sp_br_116_t_r, sr_sc_t_r, sr_sc_br_101_t_r, sr_sc_br_116_t_r]

#MODELO DE ML DE CLASSIFICAÇÃO

**Bases de dados que serão utilizadas**

O base de dados de treinamento é composta pelos registros dos acidentes ocorridos nas rodovias federais brasilerias a partir do ano de 2017 até o ano de 2021
> df_treinamento_r = [sr_df_r, sr_df_br_020_r, sr_df_br_040_r, sr_sp_r, sr_sp_br_101_r,sr_sp_br_116_r, sr_sc_r, sr_sc_br_101_r, sr_sc_br_116_r]

O base de dados de teste é composta pelos registros dos acidentes ocorridos nas rodovias federais brasilerias no ano de 2022

> df_teste_r = [sr_df_t_r, sr_df_br_020_t_r, sr_df_br_040_t_r,sr_sp_t_r, sr_sp_br_101_t_r, sr_sp_br_116_t_r, sr_sc_t_r, sr_sc_br_101_t_r, sr_sc_br_116_t_r]

In [ ]:
def executar_modelos_classificacao(bases_treinamento, bases_teste):
    modelos = {
        "Decision Tree Classifier": DecisionTreeClassifier(),
        "Linear SVC": LinearSVC(max_iter=1000, tol=1e-5),
        "Bagging Classifier": BaggingClassifier(),
        "SVC": SVC(),
        "Random Forest Classifier": RandomForestClassifier(),
        "Logistic Regression": LogisticRegression(solver='saga', max_iter=1000, C=0.1),
        "AdaBoost Classifier": AdaBoostClassifier()
    }

    scaler = StandardScaler()  # Criar um objeto scaler uma vez

    for i, (base_treinamento, base_teste) in enumerate(zip(bases_treinamento, bases_teste)):
        print(f"\nExecutando modelos para a base de treinamento {i + 1}:")
        imprimir_nome_base(base_treinamento)

        X_train = base_treinamento.drop(columns=['km', 'regional'])
        y_train = base_treinamento['km']

        X_test = base_teste.drop(columns=['km', 'regional'])
        y_test = base_teste['km']

        # Escalar os dados de treinamento
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)

        # Escalar os dados de teste usando o mesmo scaler
        X_test_scaled = scaler.transform(X_test)

        for nome_modelo, modelo in modelos.items():
            print(f"\n{nome_modelo}:")
            modelo.fit(X_train_scaled, y_train)  # Usar dados de treinamento escalados
            y_pred = modelo.predict(X_test_scaled)  # Usar dados de teste escalados
            accuracy = accuracy_score(y_test, y_pred)
            print(f"Acurácia: {accuracy}")

# Função para imprimir o nome da base de dados
def imprimir_nome_base(df):
    base_name = [name for name, var in globals().items() if var is df][0]
    print(f"Base de dados: {base_name}")

# Exemplo de uso:
executar_modelos_classificacao(df_treinamento_r, df_teste_r)



Executando modelos para a base de treinamento 1:
Base de dados: sr_df_r

Decision Tree Classifier:
Acurácia: 0.04665203073545554

Linear SVC:


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Acurácia: 0.06750823271130625

Bagging Classifier:
Acurácia: 0.05378704720087816

SVC:
Acurácia: 0.07354555433589462

Random Forest Classifier:
Acurácia: 0.052689352360043906

Logistic Regression:
Acurácia: 0.05982436882546652

AdaBoost Classifier:
Acurácia: 0.06092206366630077

Executando modelos para a base de treinamento 2:
Base de dados: sr_df_br_020_r

Decision Tree Classifier:
Acurácia: 0.029166666666666667

Linear SVC:


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Acurácia: 0.052083333333333336

Bagging Classifier:
Acurácia: 0.04375

SVC:
Acurácia: 0.0375

Random Forest Classifier:
Acurácia: 0.05

Logistic Regression:
Acurácia: 0.020833333333333332

AdaBoost Classifier:
Acurácia: 0.004166666666666667

Executando modelos para a base de treinamento 3:
Base de dados: sr_df_br_040_r

Decision Tree Classifier:
Acurácia: 0.07651245551601424

Linear SVC:


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Acurácia: 0.10676156583629894

Bagging Classifier:
Acurácia: 0.06405693950177936

SVC:
Acurácia: 0.08718861209964412

Random Forest Classifier:
Acurácia: 0.07473309608540925

Logistic Regression:
Acurácia: 0.10320284697508897

AdaBoost Classifier:
Acurácia: 0.005338078291814947

Executando modelos para a base de treinamento 4:
Base de dados: sr_sp_r

Decision Tree Classifier:
Acurácia: 0.027274810909924365

Linear SVC:


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Acurácia: 0.043089617235846894

Bagging Classifier:
Acurácia: 0.022461608984643593

SVC:
Acurácia: 0.00412560165024066

Random Forest Classifier:
